# Computation Intelligence for Optimization | Sports League Optimization
`Group AM`
- Eduardo Mendes, 20240850
- Helena Duarte, 20240530
- João Freire, 20240528
- Mariana Sousa, 20240516

<div class="alert alert-block alert-info">

# Table of Contents
    
[1. Import Libraries](#1)<br>

[2. Load data](#2)<br>

<a class="anchor" id="1">

# 1. Import Libraries
    
</a>

In [ ]:
import sys
sys.path.append(".")

# Data Preprocessing Tools
import pandas as pd
import numpy as np

# Custom Libraries
from Model.Solution import SportsLeagueSolution
from Operators.Selection import tournament_selection, ranking_selection
from Operators.Crossover import standard_crossover_with_position_repair, crossover_by_position_dual_any
from Operators.Mutation import player_swap_mutation, role_shuffle_mutation, player_role_left_shift_mutation
from Model.genetic_algorithm import SportsLeagueGASolution, genetic_algorithm


<a class="anchor" id="3">

# 2. Problem Definiton
    
</a>

In a fantasy sports league, the objective is to assign players to teams in a way that ensures
a balanced distribution of talent while staying within salary caps.

1) Each player is defined by the following attributes:
* Skill rating: Represents the player's ability.
* Cost: The player's salary.
* Position (One of four roles) : Goalkeeper (GK), Defender (DEF), Midfielder (MID), or Forward (FWD).

A solution is a complete league configuration, specifying the team assignment for each player. These are the constraints that must be verified in every solution of the search space (no object is considered a solution if it doesn’t comply with these):
* Each team must consist of: 1 Goalkeeper, 2 Defenders, 2 Midfielders and 2
Forwards.
* Each player is assigned to exactly one team.

*Impossible Configurations*: Teams that do not follow this exact structure (e.g., a team with 2 goalkeepers, or a team where the same defender is assigned twice) are not part of the search space and are not considered solutions. It is forbidden to generate such an arrangement during evolution.

Besides that, each team should not exceed a 750€ million total budget. If it does, it is not a valid solution and the fitness value should reflect that.

The `objective` is to create a balanced league that complies with the constraints. 
A balanced league a is a league where the average skill rating of the players is roughly the same among the teams. 
This can be measured by the standard deviation of the average skill rating of the teams.

You can find a dataset of players with their names, position, skill rating and salary (in million €).
These players should be distributed across 5 teams of 7 players each.

<a class="anchor" id="3">

# 3. Data Preprocessing
    
</a>

In order to use the code, some processing must be done to the dataset to re-name columns into more intuitive names and to order players by position

In [2]:
df = pd.read_csv("Data/players(in).csv", index_col=0)

# change the column names to lowercase
df.rename(columns={
    "Name": "name",
    "Position": "position",
    "Skill": "skill",
    "Salary (€M)": "salary"
}, inplace=True)

position_order = ["GK", "DEF", "MID", "FWD"] # the order of positions

# Create a mapping dictionary: {'GK': 0, 'DEF': 1, 'MID': 2, 'FWD': 3}
df_sorted = df.sort_values(by="position", key=lambda x: x.map({pos: i for i, pos in enumerate(position_order)})).reset_index(drop=True)

# Create a column "id" in the sorted DataFrame
df_sorted["id"]= df_sorted.index

df_sorted.head(5)

,name,position,skill,salary,id
0,Alex Carter,GK,85,90,0
1,Jordan Smith,GK,88,100,1
2,Ryan Mitchell,GK,83,85,2
3,Chris Thompson,GK,80,80,3
4,Blake Henderson,GK,87,95,4


<a class="anchor" id="3">

# 4. Tests
    
</a>

Tests performed to ensure the developped code works

## SportsLeagueSolution tests

In [3]:
sol = SportsLeagueSolution(players_df=df_sorted)


In [4]:
sol.repr

[[0, 9, 5, 17, 21, 26, 27],
 [2, 8, 13, 20, 24, 29, 31],
 [3, 6, 14, 18, 22, 34, 30],
 [4, 12, 10, 19, 16, 25, 33],
 [1, 11, 7, 23, 15, 32, 28]]

In [5]:
sol.fitness()

1000000000.0

## Selector Tests

In [6]:
POP_SIZE = 50
initial_population = [
    SportsLeagueSolution(players_df=df_sorted)
    for _ in range(POP_SIZE)
]

In [7]:
selected = tournament_selection(initial_population)
print("Tournament-selected fitness:", selected.fitness())

selected = ranking_selection(initial_population)
print("Ranking-selected fitness:", selected.fitness())


Tournament-selected fitness: 0.991391518451282
Ranking-selected fitness: 1.165490174535491


## Crossover Tests

In [8]:
parent1 = SportsLeagueSolution(players_df=df_sorted)
parent2 = SportsLeagueSolution(players_df=df_sorted)

In [9]:
child_repr1, child_repr2 = standard_crossover_with_position_repair(parent1.repr, parent2.repr, df_sorted)

child1 = SportsLeagueSolution(repr=child_repr1, players_df=df_sorted)
child2 = SportsLeagueSolution(repr=child_repr2, players_df=df_sorted)

print("Child 1 fitness:", child1.fitness())
print("Child 2 fitness:", child2.fitness())


Child 1 fitness: 0.7194101892579559
Child 2 fitness: 1000000000.0


In [10]:
child_repr1, child_repr2 = crossover_by_position_dual_any(parent1.repr, parent2.repr, df_sorted)

child1 = SportsLeagueSolution(repr=child_repr1, players_df=df_sorted)
child2 = SportsLeagueSolution(repr=child_repr2, players_df=df_sorted)

print("Child 1 fitness:", child1.fitness())
print("Child 2 fitness:", child2.fitness())


Child 1 fitness: 1000000000.0
Child 2 fitness: 1.1619828232950167


## Mutation Tests

In [11]:
parent = SportsLeagueSolution(players_df=df_sorted)


In [12]:
mutated = player_swap_mutation(parent, verbose=True)
print("Fitness after player swap:", mutated.fitness())

print("-----------------------------------------")

mutated = role_shuffle_mutation(parent, verbose=True)
print("Fitness after role shuffle:", mutated.fitness())

print("-----------------------------------------")

mutated = player_role_left_shift_mutation(parent, verbose=True)
print("Fitness after role left shift:", mutated.fitness())


Swapping player 13 from team 2 with player 11 from team 3
Fitness after player swap: 1.1549361675090108
-----------------------------------------
Shuffling players in role FWD, corresponding to indexes [5, 6]
Fitness after role shuffle: 1000000000.0
-----------------------------------------
Shifting role group DEF, corresponding to indexes [1, 2], by 4 positions
Fitness after role left shift: 1000000000.0


## Algorithm Tests

In [15]:
best_solution = genetic_algorithm(
     initial_population=initial_population,
     selection_algorithm=tournament_selection,
     max_gen=100,
     maximization=False,
     verbose=True,
     elitism=True,) 

-------------- Generation: 1 --------------
Selected individuals:
[[0, 9, 14, 20, 18, 31, 25], [1, 12, 11, 21, 19, 28, 29], [2, 13, 10, 24, 16, 33, 26], [4, 7, 8, 17, 22, 27, 30], [3, 6, 5, 15, 23, 32, 34]]
[[3, 6, 13, 20, 18, 25, 33], [1, 5, 8, 24, 23, 26, 32], [0, 7, 12, 15, 21, 30, 29], [4, 11, 14, 17, 22, 27, 34], [2, 9, 10, 16, 19, 31, 28]]
Applied crossover
Offspring:
[[0, 7, 5, 20, 18, 30, 25], [1, 12, 8, 21, 23, 32, 29], [3, 13, 6, 24, 15, 33, 26], [4, 11, 14, 17, 22, 27, 34], [2, 9, 10, 16, 19, 31, 28]]
[[2, 10, 13, 20, 18, 25, 33], [1, 9, 14, 24, 16, 26, 28], [0, 11, 12, 19, 21, 31, 29], [4, 7, 8, 17, 22, 27, 30], [3, 6, 5, 15, 23, 32, 34]]
First mutated individual: [[0, 7, 5, 20, 18, 30, 25], [1, 12, 8, 21, 23, 32, 29], [3, 13, 6, 24, 15, 33, 26], [4, 11, 14, 17, 22, 27, 34], [2, 9, 10, 16, 19, 31, 28]]
Second mutated individual: [[2, 10, 13, 20, 18, 25, 33], [1, 9, 14, 24, 16, 26, 28], [0, 11, 12, 19, 21, 31, 29], [4, 7, 8, 17, 22, 27, 30], [3, 6, 5, 15, 23, 32, 34]]
Select

KeyboardInterrupt: 

In [ ]:
print("Best solution:", best_solution[0])
print("Fitness:", best_solution[1][-1])

<a class="anchor" id="3">

# 5. Performance Tests
    
</a>